In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
!pip install catboost

In [30]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.ensemble import GradientBoostingClassifier

from IPython.display import Image
from IPython.core.display import HTML 

In [4]:
df = pd.read_csv("/content/gdrive/MyDrive/datasets/data.csv")

In [5]:
df.head()

,text,label
0,&#34;В регионе стало меньше пахнуть Россией&#3...,не по теме
1,&#34;Северному потоку-2&#34; предсказали полуп...,мнения
2,10 лет назад протестующие заняли площадь Тахри...,не по теме
3,130 парков для умных. В регионах открываются в...,не по теме
4,170 лыжников эвакуировали с канатной дороги в ...,не по теме


In [6]:
df.shape

(2156, 2)

In [7]:
df.isna().sum()

text     0
label    0
dtype: int64

In [8]:
print(df.label.unique())

['не по теме' 'мнения' 'аналитика' 'торговля' 'Международные отношения'
 'политика' 'COVID-19' 'военная тематика' 'проекты' 'Россия'
 'Социологические опросы' 'меры поддержки' 'инвестиция']


In [9]:
print(df.label.nunique())

13


In [10]:
df.label.value_counts()

не по теме                 891
политика                   357
COVID-19                   249
Международные отношения    177
мнения                     131
военная тематика           112
аналитика                  101
торговля                    72
меры поддержки              31
проекты                     12
Россия                      10
инвестиция                   9
Социологические опросы       4
Name: label, dtype: int64

In [11]:
train, test = train_test_split(df, test_size=0.1, shuffle=True)

In [12]:
train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

In [13]:
vectorizer = CountVectorizer(max_features=1000, min_df=10, max_df=0.1, ngram_range=(1, 3))
X = vectorizer.fit_transform(train.text)
X_test = vectorizer.transform(test.text) 

In [14]:
X.shape, X_test.shape

((1940, 1000), (216, 1000))

In [15]:
y = train.label.values
y_test = test.label.values

# LogisticRegression

In [16]:
clf = LogisticRegression(solver='lbfgs', max_iter=1000)

In [17]:
clf.fit(X, y)

LogisticRegression(max_iter=1000)

In [18]:
clf.classes_


array(['COVID-19', 'Международные отношения', 'Россия',
       'Социологические опросы', 'аналитика', 'военная тематика',
       'инвестиция', 'меры поддержки', 'мнения', 'не по теме', 'политика',
       'проекты', 'торговля'], dtype=object)

In [19]:
preds = clf.predict(X_test)

In [20]:
print(classification_report(y_test, preds, zero_division=0))

                         precision    recall  f1-score   support

               COVID-19       0.70      0.67      0.68        24
Международные отношения       0.29      0.23      0.26        26
              аналитика       0.18      0.22      0.20         9
       военная тематика       0.40      0.22      0.29         9
             инвестиция       0.00      0.00      0.00         1
         меры поддержки       0.33      0.33      0.33         3
                 мнения       0.22      0.20      0.21        10
             не по теме       0.74      0.79      0.76        85
               политика       0.57      0.64      0.61        42
                проекты       0.00      0.00      0.00         1
               торговля       0.20      0.17      0.18         6

               accuracy                           0.57       216
              macro avg       0.33      0.32      0.32       216
           weighted avg       0.56      0.57      0.56       216



# RandomForestClassifier

In [21]:
rf = RandomForestClassifier(n_estimators=100, max_depth=20)
rf.fit(X, y)

preds = rf.predict(X_test)

In [22]:
print(classification_report(y_test, preds, zero_division=0))

                         precision    recall  f1-score   support

               COVID-19       0.77      0.83      0.80        24
Международные отношения       0.17      0.04      0.06        26
              аналитика       0.33      0.11      0.17         9
       военная тематика       0.00      0.00      0.00         9
             инвестиция       0.00      0.00      0.00         1
         меры поддержки       0.00      0.00      0.00         3
                 мнения       0.00      0.00      0.00        10
             не по теме       0.50      0.92      0.65        85
               политика       0.68      0.40      0.51        42
                проекты       0.00      0.00      0.00         1
               торговля       0.00      0.00      0.00         6

               accuracy                           0.54       216
              macro avg       0.22      0.21      0.20       216
           weighted avg       0.45      0.54      0.46       216



# DecisionTreeClassifier

In [23]:
clf = DecisionTreeClassifier(max_depth=20)
clf.fit(X, y)
preds = clf.predict(X_test)

print(classification_report(y_test, preds, zero_division=0))

                         precision    recall  f1-score   support

               COVID-19       0.65      0.62      0.64        24
Международные отношения       0.43      0.23      0.30        26
                 Россия       0.00      0.00      0.00         0
              аналитика       0.17      0.11      0.13         9
       военная тематика       0.00      0.00      0.00         9
             инвестиция       0.00      0.00      0.00         1
         меры поддержки       0.50      0.33      0.40         3
                 мнения       0.00      0.00      0.00        10
             не по теме       0.63      0.86      0.73        85
               политика       0.48      0.50      0.49        42
                проекты       0.00      0.00      0.00         1
               торговля       0.20      0.17      0.18         6

               accuracy                           0.55       216
              macro avg       0.25      0.24      0.24       216
           weighted avg

# KNeighborsClassifier

In [24]:
clf = KNeighborsClassifier(n_neighbors=10, metric='cosine')
clf.fit(X, y)
preds = clf.predict(X_test)

print(classification_report(y_test, preds, zero_division=0))

                         precision    recall  f1-score   support

               COVID-19       0.64      0.75      0.69        24
Международные отношения       0.33      0.12      0.17        26
              аналитика       0.38      0.33      0.35         9
       военная тематика       0.40      0.44      0.42         9
             инвестиция       0.00      0.00      0.00         1
         меры поддержки       0.00      0.00      0.00         3
                 мнения       0.19      0.30      0.23        10
             не по теме       0.64      0.78      0.70        85
               политика       0.57      0.48      0.52        42
                проекты       0.00      0.00      0.00         1
               торговля       0.17      0.17      0.17         6

               accuracy                           0.55       216
              macro avg       0.30      0.31      0.30       216
           weighted avg       0.52      0.55      0.52       216



# MultinomialNB

In [25]:
clf = MultinomialNB(alpha=1.)
clf.fit(X, y)
preds = clf.predict(X_test)

print(classification_report(y_test, preds, zero_division=0))

                         precision    recall  f1-score   support

               COVID-19       0.46      0.71      0.56        24
Международные отношения       0.17      0.04      0.06        26
                 Россия       0.00      0.00      0.00         0
 Социологические опросы       0.00      0.00      0.00         0
              аналитика       0.20      0.22      0.21         9
       военная тематика       0.24      0.67      0.35         9
             инвестиция       0.00      0.00      0.00         1
         меры поддержки       0.00      0.00      0.00         3
                 мнения       0.00      0.00      0.00        10
             не по теме       0.85      0.27      0.41        85
               политика       0.61      0.33      0.43        42
                проекты       0.08      1.00      0.15         1
               торговля       0.11      0.83      0.20         6

               accuracy                           0.32       216
              macro avg

# LinearSVC

In [26]:
clf = LinearSVC()
clf.fit(X, y)
preds = clf.predict(X_test)

print(classification_report(y_test, preds, zero_division=0))

                         precision    recall  f1-score   support

               COVID-19       0.55      0.67      0.60        24
Международные отношения       0.30      0.23      0.26        26
                 Россия       0.00      0.00      0.00         0
              аналитика       0.30      0.33      0.32         9
       военная тематика       0.36      0.56      0.43         9
             инвестиция       0.00      0.00      0.00         1
         меры поддержки       0.25      0.33      0.29         3
                 мнения       0.18      0.20      0.19        10
             не по теме       0.73      0.62      0.67        85
               политика       0.59      0.57      0.58        42
                проекты       0.00      0.00      0.00         1
               торговля       0.17      0.33      0.22         6

               accuracy                           0.52       216
              macro avg       0.28      0.32      0.30       216
           weighted avg

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


# CatBoostClassifier

In [34]:
clf = CatBoostClassifier()

In [ ]:
clf.fit(X, y)
preds = clf.predict(X_test)

In [36]:
print(classification_report(y_test, preds, zero_division=0))

                         precision    recall  f1-score   support

               COVID-19       0.77      0.83      0.80        24
Международные отношения       0.42      0.19      0.26        26
              аналитика       0.33      0.33      0.33         9
       военная тематика       0.67      0.22      0.33         9
             инвестиция       0.00      0.00      0.00         1
         меры поддержки       0.00      0.00      0.00         3
                 мнения       0.67      0.20      0.31        10
             не по теме       0.62      0.89      0.73        85
               политика       0.54      0.50      0.52        42
                проекты       0.00      0.00      0.00         1
               торговля       0.50      0.17      0.25         6

               accuracy                           0.60       216
              macro avg       0.41      0.30      0.32       216
           weighted avg       0.57      0.60      0.56       216



In [39]:
clf.save_model('catboost_model.dump')

# GradientBoostingClassifier

In [31]:
clf = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100,max_depth=3, min_samples_split=2, min_samples_leaf=1, subsample=1,max_features='sqrt', random_state=10)

In [32]:
clf.fit(X, y)
preds = clf.predict(X_test)

In [33]:
print(classification_report(y_test, preds, zero_division=0))

                         precision    recall  f1-score   support

               COVID-19       0.80      0.83      0.82        24
Международные отношения       0.40      0.15      0.22        26
              аналитика       0.33      0.33      0.33         9
       военная тематика       0.67      0.22      0.33         9
             инвестиция       0.00      0.00      0.00         1
         меры поддержки       0.00      0.00      0.00         3
                 мнения       0.00      0.00      0.00        10
             не по теме       0.56      0.89      0.69        85
               политика       0.57      0.38      0.46        42
                проекты       0.00      0.00      0.00         1
               торговля       0.00      0.00      0.00         6

               accuracy                           0.56       216
              macro avg       0.30      0.26      0.26       216
           weighted avg       0.51      0.56      0.50       216



# Выводы:
1. Задача представляет собой многоклассовую классификацию. Для ее выполнения были выбраны классические алгоритмы машинного обучения, такие как логистическая регрессия, метод случайного леса, метод решающих деревьев, метод ближайших соседей, метод наивного байеса, метод опорных векторов и 2 вида градиентного бустинга. Выбор обсуловлен тем, что у нас не так много данных и данные алгоритмы быстро обучаются и могут служить хорошим бейзлайном для дальнейшей работы над улучшением. В качестве метрик оценки качества моделей использовались классические для данной задачи точность, полнота, F-мера и их усредненные показатели для задачи многоклассовой классификации.  

2. Из всех алгоритмов без особого тюнинга лучше всего показал себя градиентный бустинг CatBoostClassifier.

3. Чтобы улучшить результаты можно попробовать тюнинг параметров через GridSearch. Также можно попробовать другие варианты векторизации (TfIdf, word2vec, fasttext и тд) или посмотреть предобученные модели. Еще можно попробовать различные варианты предобработки: леммы, стемминг, более аккуратное удаление стоп-слов и пунктуации, использование нграм. Можно поэксперементировать с тегами частей речи. Конечно же также стоит собрать больше данных, особенно для тех классов, которые представленые в датасете в небольшом количестве. Если собрать больше данных, то можно попробовать глубокое обучение. 